## **Registration**

In [ ]:
#registration
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr

def lambda_handler(event, context):
    for k in range(0,5350,1):
        k=str(k)
        userid="imsi-20893"+str(k.zfill(10))
        ueID = userid
        dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
        table = dynamodb.Table('Subscribe_User')
        dynamodb_client = boto3.client('dynamodb')
        startSession = table.put_item(
            Item={
                'ue_ID': ueID,
                'Units': "4.8" #initial GU
            }
        )
      
    output = " Create Successful!!" 
    response = {'statusCode': 200, 'body' : output}
    return response
    

## **Get Subscribe_User**

In [ ]:
#get Subscribe_User 
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    ueID = event['ue_ID']
    dynamodb_resource = boto3.resource('dynamodb')
    suscribetable=dynamodb_resource.Table('Subscribe_User')
    GU_old = suscribetable.query(
        KeyConditionExpression=Key('ue_ID').eq(ueID)
    )
    for i in GU_old['Items']:
        GU_value=i['Units']
    output=GU_value
    response = {'statusCode': 200, 'body' : output}
    return response

## **Get_predict_data**

In [ ]:
#get_predict_data
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    ueID = event['ue_ID']
    Time = event['datetime']
    dynamodb_resource = boto3.resource('dynamodb')
    #uetable = dynamodb_resource.Table(ueID)
    uetable = dynamodb_resource.Table('Predict_data')
    # GU_cost = uetable.query(
    #     KeyConditionExpression=Key('datetime').eq(Time)
    # )
    Predict_GU = uetable.get_item(Key={'ueid': ueID, 'datetime': Time})
     
    # for i in GU_cost['Items']:
    #     GU_cumsume=i['Internet']
    rea= Predict_GU['Item']
    Predict_GU=rea['predictdata']
    
    output=Predict_GU
    response = {'statusCode': 200, 'body' : output}
    return response
    


## **Get_internet_data**

In [ ]:
#get_internet_data
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    ueID = event['ue_ID']
    Time = event['datetime']
    dynamodb_resource = boto3.resource('dynamodb')
    #uetable = dynamodb_resource.Table(ueID)
    uetable = dynamodb_resource.Table('ADRF')
    # GU_cost = uetable.query(
    #     KeyConditionExpression=Key('datetime').eq(Time)
    # )
    GU_cost = uetable.get_item(Key={'ueid': ueID, 'datetime': Time})
     
    # for i in GU_cost['Items']:
    #     GU_cumsume=i['Internet']
    rea=GU_cost['Item']
    GU_cumsume=rea['Internet']
    
    output=GU_cumsume
    response = {'statusCode': 200, 'body' : output}
    return response
    


## **Write Session**

In [ ]:
 
import boto3, json, os, datetime as dt
import urllib3
http = urllib3.PoolManager()
from boto3.dynamodb.conditions import Key, Attr

def getGuvalues(ueID):
    data = json.dumps({'ue_ID':ueID})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response
    
def putGU(ueID,value):
    data = json.dumps({'ue_ID':ueID,'value':value})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    return response
    
def getInternetvalues(ueID,time):
    data = json.dumps({'ue_ID':ueID,'datetime':time})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response

    
def lambda_handler(event, context):
    ueID = event['ue_ID']
    time = event['time']
    GU_old=float(getGuvalues(ueID))
    GU_consume=float(getInternetvalues(ueID,time))
    GU_new=GU_old-GU_consume
    GU_new=str(GU_new)
    putGU(ueID,GU_new)
    output=time+" [CTF]" +ueID+" Collecting Charging Event."
    response = {'statusCode': 200, 'body' : output}
    return response

## **Put_dynamodb**

In [ ]:
#put_dynamodb
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    #body = json.loads(event['body'])
    ueID = event['ue_ID']
    value= event['value']
    dynamodb_resource = boto3.resource('dynamodb')
    
    suscribetable=dynamodb_resource.Table('Subscribe_User')
    GU_old = suscribetable.query(
        KeyConditionExpression=Key('ue_ID').eq(ueID)
    )
  

    suscribetable.update_item(
    Key={ 'ue_ID': ueID,},
    UpdateExpression="set Units=:u",
    ExpressionAttributeValues={
       ':u': value
    },
    )
    
    response = {'statusCode': 200, 'body' : "put ok"}
    return response
    


## **Nchf_ConvergedChargingFunction_update**

In [ ]:
 
import boto3, json, os, datetime as dt
import urllib3
http = urllib3.PoolManager()
from boto3.dynamodb.conditions import Key, Attr

def getGuvalues(ueID):
    data = json.dumps({'ue_ID':ueID})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response

def getpredictvalue(ueID,time):
    data = json.dumps({'ue_ID':ueID,'datetime':time})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response
    
def putGU(ueID,value):
    data = json.dumps({'ue_ID':ueID,'value':value})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    return response
    
def nchf_convergedchargingfunction_update(ueID,time):
    data = json.dumps({'ue_ID':ueID,'datetime':time})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response
     
def lambda_handler(event, context):
    ueID = event['ue_ID']
    time = event['datetime']
    data_GU=float(getGuvalues(ueID))
    data_predict=float(getpredictvalue(ueID,time))
    # if data_GU<=0.0:
    data_GU=data_predict+data_GU
    data_GU=str(data_GU)
    putGU(ueID,data_GU)
    output=time+" [CHF] " +ueID+ " GU exhausted, Updating..."
    response = {'statusCode': 200, 'body' : output}
    return response
    # else:
    #     output=time+" [CHF]" +ueID+ " GU enough, no need to update."
    #     response = {'statusCode': 200, 'body' : output}
    #     return response
    # data_GU_check=float(getGuvalues(ueID))
    # if data_GU_check<=0.0:
    #     nchf_convergedchargingfunction_update(ueID, time)


## **Nchf_ConvergedChargingFunction_release**

In [ ]:
import json
import urllib3
http = urllib3.PoolManager()
import numpy as np
import boto3
def getInternetvalues(ueID,time):
    data = json.dumps({'ue_ID':ueID,'datetime':time})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response
    
def lambda_handler(event, context):
    CDR=[]
    for i in range(1,2,1):
        i=str(i)
        for j in range(0,12,1):
            j=str(j)
            j=j.zfill(2)
            time="2013-11-0"+i+" "+j+":00:00"
            for k in range(1,2,1):
                k=str(k)
                ueID="imsi-20893"+str(k.zfill(10))
                internetdata=getInternetvalues(ueID,time)
                CDR.append(internetdata)
    CDR=np.array(CDR)
    s3 = boto3.client('s3')
    s3_resource = boto3.resource('s3')
    np.save('/tmp/CDR.npy', CDR)
    s3_resource.Bucket('nadrf-datamanagement').upload_file("/tmp/CDR.npy","CDR.npy")                
    output="release sucessful!!"
    return {
                'statusCode': 200,
                'body': output
            }
